In [1]:
import pandas as pd
import spacy

from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [2]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [3]:
train, dev = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
train.shape, dev.shape

((960, 8), (240, 8))

In [5]:
df['rating'].unique()

array(['A', 'BB', 'AAA', 'BBB', 'AA+', 'BB+', 'BB-', 'A-', 'A+', 'B',
       'AA-', 'BBB+', 'BBB-', 'B-', 'AA', 'B+', 'C'], dtype=object)

In [6]:
categories = ['B', 'A', 'A-', 'A+', 'B-', 'B+']

In [7]:
dev = dev.query('rating in @categories')

In [8]:
train = train.query('rating in @categories')

In [9]:
train.shape, dev.shape

((350, 8), (80, 8))

In [10]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [11]:
nlp = spacy.load('ru_core_news_lg')

In [12]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [13]:
create_docbin(train).to_disk("train.spacy")
create_docbin(dev).to_disk("dev.spacy")

In [14]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: subsym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       43.72    0.44
  0     200          27.07       65.96    0.66
  1     400          19.15       72.90    0.73
  1     600          10.60       76.76    0.77
  2     800           5.71       79.00    0.79
  2    1000           6.17       82.23    0.82
  3    1200           3.46       80.61    0.81
  4    1400           3.07       83.18    0.83
  4    1600           2.04       83.38    0.83
  5    1800           2.03       84.10    0.84
  5    2000           1.74       83.39    0.83
  6    2200           1.73       84.23    0.84
  6    2400           1.31       83.88    0